In [35]:
import os
import random
import numpy as np
import shutil

import torch
import torch.utils.data as data
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
import torch.nn.functional as F

from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sn

from scipy.stats import norm

mpl.rcParams['figure.dpi']= 100
mpl.rcParams["savefig.dpi"] = 300

In [36]:
random.seed(42)
max_age = 101
age_limits = list(range(max_age))

test_split = 0.1
valid_split = 0.1

EPOCHS = 60
BATCH_SIZE = 16
LEARNING_RATE = 0.01

#optimizer
BETA1 = 0.9
BETA2 = 0.999

In [37]:
'''
os.mkdir("/kaggle")
os.mkdir("/kaggle/utkface/")
os.mkdir("/kaggle/utkface/train/")
os.mkdir("/kaggle/utkface/test/")
os.mkdir("/kaggle/utkface/valid/")
os.mkdir("/kaggle/workdir/")

all_classes = []
l = 0
for class_name in age_limits:
    all_classes.append(class_name)
    os.mkdir("/kaggle/utkface/train/"+str(class_name)+"/")
    os.mkdir("/kaggle/utkface/test/"+str(class_name)+"/")
    os.mkdir("/kaggle/utkface/valid/"+str(class_name)+"/")
'''

path = "UTKFace/"
for filename in os.listdir(path):
    age = int(filename.split("_")[0])
    if age in age_limits:
        class_name = str(age)
        shutil.copy2(path+filename, "/kaggle/utkface/train/"+class_name+"/"+filename)

In [38]:
for class_filename in os.listdir("/kaggle/utkface/train"):
    number_images = len(next(os.walk("/kaggle/utkface/train/"+class_filename))[2])
    number_test_images = int(test_split * number_images)
    number_valid_images = int(valid_split * number_images)
    all_images_filenames = next(os.walk("/kaggle/utkface/train/"+class_filename))[2]
    #select random test images from all images
    test_images_filenames = random.sample(all_images_filenames, number_test_images)
    #remove test images from all images
    all_images_filenames = [ img for img in all_images_filenames if img not in test_images_filenames]
    #select random valid images from all images
    valid_images_filenames = random.sample(all_images_filenames, number_valid_images)
    for x in test_images_filenames:
        shutil.move("/kaggle/utkface/train/"+class_filename+"/"+x, "/kaggle/utkface/test/"+class_filename+"/"+x)
    for x in valid_images_filenames:
        shutil.move("/kaggle/utkface/train/"+class_filename+"/"+x, "/kaggle/utkface/valid/"+class_filename+"/"+x)

# Data Augmentation

In [44]:
augmention_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(-30, 30)),
    transforms.RandomGrayscale(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5964, 0.4567, 0.3910], [0.1257, 0.1144, 0.1206])
])

transform_size_normalize = transforms.Compose([
    transforms.CenterCrop(224),
    ToTensor(),
    transforms.Normalize([0.5964, 0.4567, 0.3910], [0.1257, 0.1144, 0.1206])
])

transform_size = transforms.Compose([
    transforms.CenterCrop(224),
    ToTensor()
])


train_ds = torchvision.datasets.ImageFolder('/kaggle/utkface/train/', transform=augmention_transforms)
valid_ds = torchvision.datasets.ImageFolder('/kaggle/utkface/valid/', transform=transform_size_normalize)
test_ds = torchvision.datasets.ImageFolder('/kaggle/utkface/test/', transform=transform_size_normalize)
printing_data = torchvision.datasets.ImageFolder('/kaggle/utkface/test/', transform=transform_size)

train_dataloader = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
test_dataloader  = data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
valid_dataloader = data.DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
printing_dataloader = data.DataLoader(printing_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)

### DataLoader 조사로부터 내가 확인하고 싶었던 것은
1. X, y의 shape. (3, 244, 244) 이미지를 입력받아 스칼라값(0~100 range)을 배출한다. 이미지 전처리 방법은 아직 잘 모르겠어서 논문을 참고해봐야겠다.
 range가 명확하게 보이지 않는다.

In [45]:
def normal_pdf_age(mean, std, max_age=101):
    norm_pdf = norm(loc = mean, scale = std)
    p = torch.tensor(norm_pdf.pdf(torch.tensor(list(range(max_age)))))
    return p/torch.sum(p)

In [46]:
def Conv(in_channels, out_channels, kerner_size=3, stride=1, padding=1):
    out_channels = int(out_channels)
    in_channels = int(in_channels)
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kerner_size, stride, padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(True),
    )

class DLDLv2(nn.Module):
    def __init__(self, max_age=101, c=0.5):
        super(DLDLv2, self).__init__()
        self.conv1 = Conv(3, 64*c)
        self.conv2 = Conv(64*c, 64*c)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = Conv(64*c, 128*c)
        self.conv4 = Conv(128*c, 128*c)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = Conv(128*c, 256*c)
        self.conv6 = Conv(256*c, 256*c)
        self.conv7 = Conv(256*c, 256*c)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv8 = Conv(256*c, 512*c)
        self.conv9 = Conv(512*c, 512*c)
        self.conv10 = Conv(512*c, 512*c)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.conv11 = Conv(512*c, 512*c)
        self.conv12 = Conv(512*c, 512*c)
        self.conv13 = Conv(512*c, 512*c)
        self.HP = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.AvgPool2d(kernel_size=7, stride=1)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(int(512*c), max_age),
            nn.Sigmoid()
        )
        self.ages = torch.tensor(list(range(max_age))).t().float()
        self.device = "cpu"
        self.transform_normalize = torchvision.transforms.Compose([
            transforms.Normalize([0.5964, 0.4567, 0.3910], [0.1257, 0.1144, 0.1206])
        ])
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool4(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x) 
        x = self.HP(x)
        x = x.view((x.size(0), -1))
        x = self.fc1(x.view((x.size(0), -1)))
        x = F.normalize(x, p=1, dim=1)
        return x
    
    def to(self, device):
        module = super(DLDLv2, self).to(device)
        module.ages = self.ages.to(device)
        self.device = device
        return module

    #train model on a batch
    def train_batch(self, x, y):
        x  = x.to(device)
        b_x = Variable(x)
        outputs = self.forward(b_x)
        age = torch.matmul(outputs,self.ages)
        pdf = []
        for z in y:
            label_distributions =  normal_pdf_age(z, 5)
            pdf.append(label_distributions)
        pdf = torch.stack(pdf)
        pdf = pdf.to(device)
        y = y.to(device)
        b_y = Variable(y)
        return custom_loss(outputs, age, b_y, pdf)
    
    
    #predict age of a batch
    def predict_age(self, x):
        x = x.to(device)
        with torch.no_grad():
            outputs = self.forward(x)
        return torch.matmul(outputs,self.ages)
        
    def predict_age_normalize(self, x):
        x = self.transform_normalize(x)
        x = x.to(device)
        with torch.no_grad():
            outputs = self.forward(x)
        return torch.matmul(outputs,self.ages), outputs

In [47]:
compression_rate = 0.5
model = DLDLv2(max_age, compression_rate)

# model.load_state_dict(torch.load("/kaggle/input/convert-pretrained-thinage-to-torchmodel/ThinAgeNet-ChaLearn16.pt"))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    model.to("cuda")

optimizer = torch.optim.Adam(model.parameters(), betas=(BETA1, BETA2), lr=LEARNING_RATE)
num_training_steps = EPOCHS * len(train_dataloader)
lr_scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

#define loss functions
loss_l1_fc = nn.L1Loss(reduction='mean')
#get age tensor
ages = torch.tensor(list(range(max_age+1))[1:]).to(device).t().float()

def custom_loss(logits, age, label_age, label_distribution, y=1, max_age=101):
    batchsize = len(label_age)
    #l1 loss
    l1_loss = torch.sum(torch.abs(age-label_age))/batchsize

    #KL loss
    log_logits = torch.log(logits)
    kl_loss = (-1 * torch.sum(torch.diag(torch.matmul(log_logits, label_distribution.float().t()))))/batchsize
    
    #combined loss
    combined_loss = y * l1_loss + kl_loss
    
    return combined_loss, l1_loss

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.00 GiB total capacity; 2.60 GiB already allocated; 0 bytes free; 2.61 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [43]:
training_loss_history, test_loss_history = [], []

model.train()

print("-------------------------------")
for epoch in range(EPOCHS):
    
    #track trianing loss
    training_loss = 0
    training_loss_length = 0
    
    #track test loss
    test_loss = 0
    test_loss_length = 0

    for (x, y) in train_dataloader:
        loss, l1_loss = model.train_batch(x, y)
        
        #track loss
        training_loss += l1_loss.data
        training_loss_length += 1
        
        #back propagate and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
    
    #testing accuracy
    for (x, y) in test_dataloader:
        #predict age
        age = model.predict_age(x)
        
        #compute and track loss
        y = y.to(device)
        l1_loss = loss_l1_fc(age, y)
        
        #track loss
        test_loss += l1_loss.data
        test_loss_length += 1
        
    
    
    training_loss_history.append(training_loss/training_loss_length)
    test_loss_history.append(test_loss/test_loss_length)

    if epoch % 5 == 0:
        print("epoch:         {}".format(epoch))
        print("training loss: {:6.4f}".format(training_loss/training_loss_length))
        print("testing loss:  {:6.4f}".format(test_loss/test_loss_length))
        print("-------------------------------")

-------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 0; 3.00 GiB total capacity; 2.60 GiB already allocated; 0 bytes free; 2.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF